# <mark>*Librairies Importation* </mark>

In [2]:
#basic functionnality
import matplotlib.pyplot as plt
import numpy as np
#image processing functionnality
# import cv2 as cv
# import PIL
# import pathlib 
# import os
# import random
# import glob
#tensorflow functionnality
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten,Rescaling
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# <mark>*loading data* </mark>

In [3]:
#os.chdir('/content/drive/MyDrive/Colab Notebooks/projects/melanoma_datasets')
dataset_url="/content/drive/MyDrive/Colab Notebooks/projects/melanoma_datasets"

test_path="/content/drive/MyDrive/Colab Notebooks/projects/melanoma_datasets/test"
train_path="/content/drive/MyDrive/Colab Notebooks/projects/melanoma_datasets/train"
valid_path="/content/drive/MyDrive/Colab Notebooks/projects/melanoma_datasets/validation"

In [4]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path,target_size=(224,224),classes=['malignant','benign'],batch_size=30)
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path,target_size=(224,224),classes=['malignant','benign'],batch_size=30)
valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=valid_path,target_size=(224,224),classes=['malignant','benign'],batch_size=30)

Found 16006 images belonging to 2 classes.
Found 1605 images belonging to 2 classes.
Found 3001 images belonging to 2 classes.


# <mark>*Build the model* </mark>

In [5]:
#inceptionV3 is also known as googLeNet
scale=1./255
vgg16_model = Sequential()
pretrained_model = VGG16(
    include_top=False,
    input_shape=(224,224,3),  #googleNet model take 299x299 as resolution that why we have reshape the images
    pooling = 'max',
    weights='imagenet'
)
#i'll be using transfert learning so i'm gonna freeze the layers
for layer in pretrained_model.layers:
  layer.trainable=False
# add layers
vgg16_model.add(pretrained_model)
vgg16_model.add(layers.Rescaling(scale))
vgg16_model.add(Flatten())
vgg16_model.add(Dense(512,activation='relu'))
vgg16_model.add(Dense(256,activation='relu'))
vgg16_model.add(Dense(2,activation='sigmoid'))
vgg16_model.summary()
tb_call_back=tf.keras.callbacks.TensorBoard(log_dir="/content/drive/MyDrive/Colab Notebooks/projects/melanoma_call_backs_folders/VGG16_logs/",histogram_freq=1)
vgg16_model.compile(
      optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

58889256/58889256 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 rescaling (Rescaling)       (None, 512)               0         
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 2)                 514       
                                                         

# <mark>*Train the model* </mark>

In [ ]:
history = vgg16_model.fit(
    train_batches,
    validation_data=valid_batches,
    epochs=100,
    verbose=1,
    callbacks=[tb_call_back]
)

Epoch 1/100
 89/534 [====>.........................] - ETA: 2:38:36 - loss: 0.4422 - accuracy: 0.7959

# <mark>*Plotting the Trainning model state* </mark>

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train','validation'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['train','validation'])
plt.show()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/content/drive/MyDrive/Colab Notebooks/projects/melanoma_call_backs_folders/VGG16_logs/"